# Первый скрипт: получение и загрузка в базу исторических данных.

Интервал данных для загрузки определяется параметрами:<br>
   - HISTORICAL_START_DATE - дата, начиная с которой необходимо получить данные, указать вручную <br>
   - HISTORICAL_FINAL_DATE - прошедшие сутки, вычисляется автоматически <br>

In [1]:
import psycopg2
import configparser
from datetime import timedelta
from datetime import datetime
import requests
import json
from tqdm import tqdm

In [2]:
config = configparser.ConfigParser()
config.read("config.ini")

HOST = config["Database"]["HOST"]
DATABASE = config["Database"]["DATABASE"]
USER  = config["Database"]["USER"]
PASSWORD  = config["Database"]["PASSWORD"]
API_URL = config["API"]["api_url"]

In [3]:
HISTORICAL_START_DATE = datetime(2025, 7, 1, 0, 0, 0, 0) 
delta = datetime.today() - timedelta(days = 1)
HISTORICAL_FINAL_DATE = delta.replace(hour=23, minute=59, second=59, microsecond=999999)

In [4]:
def get_response(API_URL, params):
    r = requests.get(API_URL, params)
    if r.status_code == 200:
        r.json()
    else:
        print(f'connection failed with status_code {r.status_code}')
        
    return r.json()


def extract_response_data(response):
    search_arr = [] 
    
    for item in response:
        if item:
            if not item['passback_params']:
                continue
            else:
                part = item['passback_params'].replace('\'', '\"')
                data = json.loads(part)

                search_arr.append([
                item.get('lti_user_id', None),
                data.get('oauth_consumer_key', None),
                data.get('lis_result_sourcedid', None),
                data.get('lis_outcome_service_url', None),
                item.get('is_correct', None),
                item.get('attempt_type', None),
                item.get('created_at', None)
                ])
        else:
            continue
    return search_arr



def get_historical_data(HISTORICAL_START_DATE, HISTORICAL_FINAL_DATE):
    
    conn = psycopg2.connect(
        host = HOST,
        database = DATABASE,
        user = USER,
        password = PASSWORD)
    
    cursor = conn.cursor()

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS lms_database (
            user_id TEXT,
            oauth_consumer_key TEXT,
            lis_result_sourcedid TEXT,
            lis_outcome_service_url TEXT,
            is_correct TEXT,
            attempt_type TEXT,
            created_at TIMESTAMPTZ
        )
    ''')
    conn.commit()
    
    total_iterations = (HISTORICAL_FINAL_DATE - HISTORICAL_START_DATE).days + 1

    while HISTORICAL_START_DATE < HISTORICAL_FINAL_DATE:
        for _ in tqdm(range(total_iterations), desc="Loading data"):
            END_DATE = HISTORICAL_START_DATE + timedelta(days=1)

            if END_DATE > HISTORICAL_FINAL_DATE:
                END_DATE = HISTORICAL_FINAL_DATE

            params = {
                'client': 'Skillfactory',
                'client_key': 'M2MGWS',
                'start': HISTORICAL_START_DATE,
                'end': END_DATE
            }

            response =  get_response(API_URL, params)

            arr = extract_response_data(response)

            for row in arr:
                cursor.execute('''
                    insert into lms_database values (%s, %s, %s, %s, %s, %s, %s)
                  ''', row)
                conn.commit()


            HISTORICAL_START_DATE = END_DATE


In [5]:
get_historical_data(HISTORICAL_START_DATE, HISTORICAL_FINAL_DATE)

Loading data: 100%|████████████████████████████████████████████████████████████████████| 24/24 [14:07<00:00, 35.32s/it]
